In [2]:
!pip install transformers
!pip install sentencepiece
!pip install torchmetrics
!pip install pytorch_lightning
!pip install wandb
!wandb.login()

/bin/bash: -c: line 1: syntax error: unexpected end of file


In [32]:
import json
from functools import partial
import re
#from transformers import XLMRobertaTokenizer, XLMRobertaModel
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn
from torch.optim import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel
from pytorch_lightning.loggers import WandbLogger

from sklearn.model_selection import train_test_split

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoder = BertModel.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# encoder = AutoModel.from_pretrained('bert-base-cased')
#tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
# encoder

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
EXP_NAME = 'expi_1'
TRAIN_BATCH_SIZE = 2 
DEV_BATCH_SIZE = 64
ACCUMULATE_GRAD = 8

In [53]:
class SimilarityRegressor(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(SimilarityRegressor, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.dropout1 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(2*self.hidden_size, self.hidden_size//2)
        self.dropout2 = nn.Dropout(p=0.2)
        self.activation2 = nn.LeakyReLU(negative_slope=0.1)
        self.linear3 = nn.Linear(self.hidden_size//2, 1)
        #self.activation3 = nn.Sigmoid()

    def common_compute(self, x):
#         x = self.encoder(**x)[0][:, 0]
        x = self.encoder(x).last_hidden_state[:, 0, :]
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        return x
    
    def forward(self, x1, x2):
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        x = torch.cat([torch.abs(x1 - x2), (x1 + x2)], dim=-1)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        #x = 3*self.activation3(x) + 1

        return x

In [54]:
class LitSimilarityRegressor(pl.LightningModule):
    def __init__(self, encoder, embed_size=768, hidden_size=256):
        super(LitSimilarityRegressor, self).__init__()
        self.model = SimilarityRegressor(encoder, embed_size=embed_size, hidden_size=hidden_size)

        self.train_loss = torchmetrics.MeanMetric(compute_on_step=True)
        self.dev_loss = torchmetrics.MeanMetric(compute_on_step=False)
        self.test_loss = torchmetrics.MeanMetric(compute_on_step=False)

        self.train_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=True)
        self.dev_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)
        self.test_mape = torchmetrics.MeanAbsolutePercentageError(compute_on_step=False)

        self.train_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=True)
        self.dev_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)
        self.test_pcc = torchmetrics.PearsonCorrCoef(compute_on_step=False)

    def forward(self, x1, x2):
        return self.model(x1, x2)

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5, betas=(0.9, 0.99), eps=1e-8, weight_decay=0.01)

    def training_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def validation_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}
    
    def test_step(self, batch, batch_idx):
        x1, x2, scores = batch
        output = self(x1, x2)
        loss = F.mse_loss(input=output, target=scores)

        return {'loss': loss, 'preds': output, 'target': scores}

    def predict_step(self, batch, batch_idx):
        x1, x2, _ = batch
        output = self(x1, x2)

        return {'preds': output}

    def training_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.log('train/step/loss', self.train_loss(loss))
        self.log('train/step/mape', self.train_mape(preds, target))
        self.log('train/step/pcc', self.train_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,))))

    def validation_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.dev_loss(loss)
        self.dev_mape(preds, target)
        self.dev_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def test_step_end(self, outs):
        loss = outs['loss']
        preds = outs['preds']
        target = outs['target']

        self.test_loss(loss)
        self.test_mape(preds, target)
        self.test_pcc(torch.reshape(preds, (-1,)), torch.reshape(target, (-1,)))

    def training_epoch_end(self, outs):
        self.log('train/epoch/loss', self.train_loss)
        self.log('train/epoch/mape', self.train_mape)
        self.log('train/epoch/pcc', self.train_pcc)

    def validation_epoch_end(self, outs):
        self.log('dev/loss', self.dev_loss)
        self.log('dev/mape', self.dev_mape)
        self.log('dev/pcc', self.dev_pcc)
    
    def test_epoch_end(self, outs):
        self.log('dev/loss', self.test_loss)
        self.log('dev/mape', self.test_mape)
        self.log('dev/pcc', self.test_pcc)

In [55]:
model = LitSimilarityRegressor(encoder, embed_size=768, hidden_size=256)
# model

In [56]:
train_text = list()
train_file = open('/kaggle/input/enginh-sts/en-train.txt','r')
train_text_lines = train_file.readlines()
for line in train_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    score = float(score.strip())
    train_text.append([s1,s2,score])
val_text = list()
val_file = open('/kaggle/input/enginh-sts/en-val.txt','r')
val_text_lines = val_file.readlines()
for line in val_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    score = float(score.strip())
    val_text.append([s1,s2,score])
test_text = list()
test_file = open('/kaggle/input/englist-sts-test/test-sentences-2017.txt','r')
test_text_lines = test_file.readlines()
for line in test_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    score = float(score.strip())
    test_text.append([s1,s2,score])

In [57]:
val_text[0]

['an alarm in the offing on climatechange      extremeweather globalwarming eco',
 'an alarm in the offing on climate change',
 4.6]

In [58]:
def collate_fn(batch, tokenizer):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:

        text1 = str(sample[0]).lower().strip()
        text2 = str(sample[1]).lower().strip()
        
        score = torch.tensor([sample[2]])
        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score)

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
#     tokenized_1 = [tokenizer.encode(sent,add_special_tokens = True) for sent in texts_1]
#     tokenized_2 = [tokenizer.encode(sent,add_special_tokens = True) for sent in texts_2]
#     # Pad and truncate the tokenized sequences to a fixed length
#     max_len = 64
#     padded_1 = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq[:max_len]) for seq in tokenized_1], batch_first=True)
#     padded_2 = torch.nn.utils.rnn.pad_sequence([torch.tensor(seq[:max_len]) for seq in tokenized_2], batch_first=True)

#     attention_mask_1 = (padded_1!=0).float()
#     attention_mask_2 = (padded_2!=0).float()
    
    
    #     print("From tokerizer:",texts_1)
    scores = torch.cat(scores, dim=0).unsqueeze(1)

    return texts_1, texts_2, scores
#     return padded_1,padded_2,scores

In [59]:
collate_partial = partial(collate_fn, tokenizer=tokenizer)
dataloader = DataLoader(train_text, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_partial)
next(iter(dataloader))

({'input_ids': tensor([[  101,  1038,  1998,  1039,  2024,  2119,  3033,  1997,  3143, 25022,
          11890, 16446,  2000,  1996,  6046,   102],
         [  101,  2022,  2284,  2012,  2030,  2006,  2327,  1997,  2242,   102,
              0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])},
 {'input_ids': tensor([[  101, 25548,  1038,  1998,  1039,  2024,  2145,  1999,  2701, 10425,
           2007,  1996,  6046,   102],
         [  101,  2191,  1037,  2143,  2030,  9982,  1997,  2242,   102,     0,
              0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [60]:
class MLNSDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, dev_dataset, test_dataset, train_batch_size, dev_batch_size, collate_fn, tokenizer):
        super(MLNSDataModule, self).__init__()
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        self.train_batch_size = train_batch_size
        self.dev_batch_size = dev_batch_size
        self.collate_fn = collate_fn
        self.tokenizer = tokenizer

    def train_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.train_batch_size, collate_fn=collate_partial)

    def val_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.dev_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def test_dataloader(self):
        collate_partial = partial(self.collate_fn, tokenizer=self.tokenizer)
        return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)

    def predict_dataloader(self):
        #return DataLoader(self.test_dataset, shuffle=False, batch_size=self.dev_batch_size, collate_fn=collate_partial)
        return self.test_dataloader()

In [61]:
data_module = MLNSDataModule(train_text, val_text, test_text, TRAIN_BATCH_SIZE, DEV_BATCH_SIZE, collate_fn=collate_fn, tokenizer=tokenizer)
data_module

**Training**

In [62]:
logger = WandbLogger(save_dir=EXP_NAME, project=EXP_NAME, log_model=False)
logger

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


In [63]:
checkpoint_callback = ModelCheckpoint(
    dirpath='/kaggle/working/',
    filename='{epoch}-{step}',
    monitor='dev/pcc',
    mode='max',
    save_top_k=1,
    verbose=True,
    save_last=False,
    save_weights_only=False,
    every_n_epochs=1
)

In [64]:
trainer = pl.Trainer(max_epochs=10,
                     accumulate_grad_batches=ACCUMULATE_GRAD,
                     accelerator='gpu',
                     check_val_every_n_epoch=1, val_check_interval=0.25,
                     enable_progress_bar=True,
                     gradient_clip_val=0.25, track_grad_norm=2,
                     enable_checkpointing=True,
                     callbacks=[checkpoint_callback],
                     logger=logger,
                     enable_model_summary=True)

trainer

In [65]:
trainer.fit(model, datamodule=data_module)
#trainer.fit(model, datamodule=data_module, ckpt_path=root_path+'model/epoch=3-step=571.ckpt')

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /kaggle/working exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:218: UserWarning: strategy=ddp_spawn and num_workers=0 may result in data loading bottlenecks. Consider setting num_workers>0 and persistent_workers=True
  "strategy=ddp_spawn and num_workers=0 may result in data loading bottlenecks."


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py", line 248, in __getattr__
    return self.data[item]
KeyError: 'size'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 139, in _wrapping_function
    results = function(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 650, in _fit_impl
    self._run(model, ckpt_path=self.ckpt_path)
  File "/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 1112, in _run
    results = self._run_stage()
  File "/opt/conda/lib/python3.7/site-p

In [ ]:
test_pred = trainer.predict(datamodule=data_module)

all_outputs = list()
for batch_outputs in test_pred:
    all_outputs.append(batch_outputs['preds'])
all_outputs = torch.cat(all_outputs, dim=0)

all_outputs.shape


In [ ]:
test_result = trainer.test(datamodule=data_module)